###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2021 Lorena A. Barba

# Logistic regression

In Lesson 1 of this module, you learned about fitting a line to data (linear regression) using the method of gradient descent to find the model parameters (slope and $y$-intercept). But what if the observational data looks nothing like it has a linear relationship?

A major class of problems deals with binary classification, that is, the data belong to one of two categories. 
Typically, we code the two categories with $0$ and $1$. For example, is an email spam, or not spam? Is a credit-card transaction fraudulent, or legitimate? 
In these settings, often the data correspond to numbers between $0$ and $1$ that represent some _probability_ (e.g., that the email be spam or not).
One effective way to build a model representing this situation is logistic regression. The output of the model should be a number between $0$ and $1$, for inputs in the whole real line.

## Logistic function

We can build a model that will output a value between zero and one by making a non-linear transformation of the linear regression. 
This is achieved with the _logistic function_:

$$ \sigma(z) = \frac{1}{1+e^{-z}}$$

With the data consisting of two arrays, $x, y$, the model is a composition: $z= wx + b$ and $\sigma(z)$ is the output.
The task is finding $w$ and $b$ so that $\sigma(z)$ is closest to $y$ for all data points.

Let's play with this function using SymPy. We'll need NumPy and Matplotlib later, so might as well load all our libraries now.

In [ ]:
import sympy
import numpy

from matplotlib import pyplot
%matplotlib inline

In [ ]:
z = sympy.Symbol('z', real=True)

logistic = 1/(1+ sympy.exp(-z))
logistic

In [ ]:
sympy.plotting.plot(logistic);

That's a well-groomed $S$-shaped function: it's called a _sigmoid_ curve. Notice that when $z=0$ it takes the value $0.5$, and it approaches zero on the left, and one on the right.

Let's generate some synthetic data to play with. (We take this example from the SciPy 2019 tutorial by Eric Ma [1]). 
Our goal is to use gradient descent to find the model parameters $w$ and $b$ that best fit the data, in some sense that we need to discover.

In [ ]:
# synthetic data
x_data = numpy.linspace(-5, 5, 100)
w = 2
b = 1
z_data = w * x_data + b + numpy.random.normal(size=len(x_data))
y_data = 1 / (1+ numpy.exp(-z_data))

pyplot.scatter(x_data, y_data, alpha=0.4);

## Logistic loss function

To use gradient descent, we need a loss function that we can optimize with respect to the parameters. 
If we were to use a mean-square-error loss function, like in linear regression, we run into a few problems. Let's discuss.

Taking the derivatives to optimize would involve $\sigma^{\prime}(z)$, so let's look at that.

In [ ]:
lprime = logistic.diff(z)
lprime

In [ ]:
sympy.plotting.plot(lprime);

The derivative of the logistic function takes very small values at the long tails on each side of $z=0$. 
If our loss function has a $\sigma^{\prime}(z)$ factor (coming from the chain rule), this would lead to _slow learning_. 
We'd like to work with a better loss function, that avoids this problem, and we build one below by integration. (For a more detailed discussion, we recommend Chapter 3 of Michael Nielsen's free ebook [2]).

It's important to note also that our prediction model is a nonlinear function, composed with the linear model, and the square-error would lead to a non-convex loss function that can have local minima, and make gradient descent fail. 
Here's an example posted on Stackoverflow in answer to this very [question](https://math.stackexchange.com/questions/2381724/logistic-regression-when-can-the-cost-function-be-non-convex). Consider just three data points, and a model with no intercept, $z = wx$: $(-1, 2), (-20, -1), (-5, 5)$. What would a square-error mean function look like? We can plot it using SymPy.

In [ ]:
badloss = (1/(1+ sympy.exp(-z))-2)**2 + \
          (1/(1+ sympy.exp(-20*z))+1)**2  + \
          (1/(1+ sympy.exp(-5*z))-5)**2
badloss

In [ ]:
sympy.plotting.plot(badloss, xlim=(-1,1));

You can see a local minimum, where gradient descent could get stuck!

But let's go back to inspecting the derivative of the logistic function. 
We note that $\sigma^{\prime}(z)$ has the same expression in the denominator as $\sigma(z)$, but squared. Let's play around with this...

In [ ]:
lprime/logistic

OK, we can try to express this in terms of $\sigma(z)$: add and subtract $1$ to the numerator and factor:

$$\frac{e^{-z}}{1+e^{-z}} = \frac{1+e^{-z}-1}{1+e^{-z}} = 1 - \frac{1}{1+ e^{-z}} = 1 - \sigma(z)$$

We get this interesting property for the derivative of the logistic function:

$$\sigma^{\prime}(z) = \sigma(z) (1-\sigma(z))$$

Following Nielsen [2], let's call $\sigma(z)=a$, representing the predicted value by the model where $z=wx +b$. 
The deviation between the model output and the observational data at each point is measured by $|y-a|$. 
A square-error loss function, as used for linear regression, is $(y-a)^2$, giving derivatives with respect to the parameters that are proportional to the error: 
the larger the error, the larger the step taken in gradient descent. 
That's a nice property.

Seeking a loss function $L$ with the same behavior, we would like, for example:
$\frac{\partial L}{\partial b} = (y-a)$. 
Using the chain rule,
$$\frac{\partial L}{\partial b} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial z} = \frac{\partial L}{\partial a} \sigma^{\prime}(z) = \frac{\partial L}{\partial a} \, a(1-a)$$

using the property of $\sigma^{\prime}$ in the last step.

Our desired behavior for $L$ is that this derivative be $(y-a)$, giving:

$$ \frac{\partial L}{\partial a} = \frac{(y-a)}{a(1-a)}$$


This lets us construct $L$ by integration:

In [ ]:
a, y = sympy.symbols('a y', real=True)

In [ ]:
dLda = (y-a)/a/(1-a)
dLda

In [ ]:
L = sympy.integrate(dLda, a)
L

In [ ]:
sympy.simplify(L)

(Plus a constant of integration.) We note that since $a<1$ (and the logarithm is only defined for positive real numbers) we need:

In [ ]:
L = -y*sympy.log(a) + (y-1)*sympy.log(1-a)
L

In [ ]:
sympy.simplify(L.diff(a))

The derivative is the same expression we started with. 
This is the _logistic loss function_ for a single data point:

$$L = -y \log(a) + (y-1) \log(1-a)$$

Let's inspect this loss function. If the true value is $y=1$, we get $L= -\log(a)$. 
Plotting in the range of interest shows that the gradient is largest when the model output is furthest away from the true value. 
This is a nice property.

In [ ]:
sympy.plotting.plot(-sympy.log(a), xlim=(0,1));

If $y=0$: $L= - \log(1-a)$. The plot again shows that the gradient is largest when the model output is furthest from the true value.

In [ ]:
sympy.plotting.plot(-sympy.log(1-a), xlim=(0,1));

## Find the parameters using `autograd`

With the logistic loss function, we're ready to use gradient descent in an optimization loop. 
Above, we defined the `logistic` SymPy expression as a function of the symbol `z`, so let's re-define it to include the composition with $wx+b$. 
We could then define the loss function symbolically, as we did for linear regression; have a look at this:

In [ ]:
logistic

In [ ]:
w, b, x, y = sympy.symbols('w b x y')
logistic = 1/(1+ sympy.exp(-w*x-b)) # redefined with the composition

Loss = -y*sympy.log(logistic) - (1-y)*sympy.log(1-logistic)
Loss

You can imagine that it will be rather messy to use SymPy for obtaining the derivatives of this loss function symbolically, then transforming them into Python functions, and then writing a gradient descent loop. 
Have a look at the derivative with respect to the parameter $b$: 

In [ ]:
Loss.diff(b)

There's a better way! It's called _automatic differentiation_: the idea is to algorithmically obtain derivatives of numeric functions written in computer code. 
It sounds like magic, but it can be done by a combination of the chain rule, symbolic rules of differentiation for elementary operations, and a numeric evaluation trace of the elementary derivatives. 

The details of how automatic differentiation works are fascinating, but here we skip ahead to say that software libraries are freely available to do this! 
For Python programs, the [`autograd`](https://github.com/hips/autograd) package was specifically developed for gradient-based optimization. 
It is described thus:

> Autograd can automatically differentiate native Python and NumPy code. It can handle a large subset of Python's features, including loops, ifs, recursion and closures, and it can even take derivatives of derivatives of derivatives.

(The package is no longer in active development, as a new version was folded into the [JAX](https://github.com/google/jax) project, out of Google. It is still maintained, though, and just as useful as ever!)

To install the package, you can run the following command (either on the terminal or in a Jupyter code cell):

```
pip install autograd
```

or, if you installed your Python environment via Anaconda, you can run this command:
```
conda install -c conda-forge autograd
```

Once the package is installed in your working environment, you can import it in the usual Pythonic way. 
The main function you will need is `autograd.grad()`, which takes a scalar-valued Python function as argument, and returns another function that evaluates to its derivative. 
It's what we use in the optimization loop to perform gradient descent.

In addition, `autograd.numpy` is a wrapper to the NumPy library. This allows you to call your favorite NumPy methods with `autograd` keeping track of every operation so it can give you the derivative (via the chain rule).
We ill import it using the alias (`as np`), consistent with the tutorials and documentation that you will find online.
Up to now in the _Engineering Computations_ series of modules, we have refrained from using the aliased form of the import statements, just to have more explicit and readable code. 

In [ ]:
# import the autograd-wrapped version of numpy
import autograd.numpy as np

In [ ]:
# import the gradient calculator
from autograd import grad  

It's important to realize that we now need to design our program in a _functional_ style: defining Python functions for the logistic and loss functions, so that we may apply `autograd.grad()` in the optimization loop. 
Study the code below.

In [ ]:
# note: the namespace np is the autograd wrapper to NumPy

def logistic(z):
    '''The logistic function'''
    return 1 / (1 + np.exp(-z))
    
def logistic_model(params, x):
    '''A prediction model based on the logistic function composed with wx+b
       params: array(w,b) of model parameters
       x :  array of x data'''
    w = params[0]
    b = params[1]
    z = w * x + b
    y = logistic(z)
    return y

def log_loss(params, model, x, y):
    '''The logistic loss function'''
    y_pred = model(params, x)
    return -np.mean(y * np.log(y_pred) + (1-y) * np.log(1 - y_pred))

In [ ]:
# get a function to compute the gradient of the logistic loss
gradient = grad(log_loss)

In [ ]:
type(gradient)

##### Note:

> The argument list of the function returned by `grad()` will be the same as the argument list of the loss function that we input to it.

Let's now make a random starting guess for the two model parameters:

In [ ]:
params = np.random.rand(2)
print(params)

With values assigned to the two parameters, this is how you compute the corresponding derivatives—we'll need to do this in each iteration of the optimization loop:

In [ ]:
gradient(params, logistic_model, x_data, y_data)

Now we optimize! Notice that we set both a maximum number of iterations, and an exit criterion based on two successive residuals being very close. 

Finally, we plot the synthetic data we created above, and the logistic regression curve corresponding to the parameters found in the optimization loop.

In [ ]:
for i in range(3000):
    descent = gradient(params, logistic_model, x_data, y_data)
    oldparams = params
    params = params - descent * 0.01
    residual = np.abs((params - oldparams) / oldparams)
    if np.all(residual < 1e-6):
        break

print(f'Optimized value of w is {params[0]:.3f} vs. true value: 2')
print(f'Optimized value of b is {params[1]:.3f} vs. true value: 1')
print(f'Exited after {i} iterations')
print(f'Residual is {residual}')

pyplot.scatter(x_data, y_data, alpha=0.4)
pyplot.plot(x_data, logistic_model(params, x_data), '-r');

It worked! We found a logistic curve that approximates the data quite well. 
This curve represents the _prediction model_ for the continuous data between $0$ and $1$. 
In a classification task, the next step is to assign a decision threshold for the two classes; a natural choice is $0.5$.

In [ ]:
def decision_boundary(y):
    return 1 if y >= .5 else 0

This function will code as $0$ or $1$ each data point. We will want to apply the function element-wise to all elements in an array, for which we can use [`numpy.vectorize()`](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html). 
Be sure to read the documentation of this very useful NumPy function that saves you from having to write a for-loop!

The decision rule is what turns logistic regression into a classification tool. Check it out!

In [ ]:
decision_boundary = numpy.vectorize(decision_boundary)

In [ ]:
def classify(predictions):
    '''
    Argument:
    predictions, an array of values between 0 and 1
    
    Returns: 
    classified, an array of 0 and 1 values'''

    return decision_boundary(predictions).flatten()

pyplot.scatter(x_data, classify(x_data), alpha=0.4);

And this is the end of Lesson 2 of our module. We spent some time deriving the logistic loss function, where most sources will just give it to you without explanation. 
We find it more satisfying this way, but you can certainly skip it. 

The key ingredients we have added in this lesson are the ideas of:
- using composition of functions in our model
- using automatic differentiation to get the gradients for the optimization loop

In the next lesson, we add the idea of multiple independent variables (a.k.a. _features_) affecting our model, leading to **multiple linear regression.**

## References

1. Eric Ma, "Deep Learning Fundamentals: Forward Model, Differentiable Loss Function & Optimization," SciPy 2019 tutorial. [video on YouTube](https://youtu.be/JPBz7-UCqRo) and [archive on GitHub](https://github.com/ericmjl/dl-workshop/releases/tag/scipy2019).
2. Michael A. Nielsen, "Neural Networks and Deep Learning" (2015), Determination Press, http://neuralnetworksanddeeplearning.com

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())